In [38]:
import os
import numpy as np
from tqdm import tqdm

from src.feature_extraction import wav_to_logmel


In [39]:
import os
from pathlib import Path

# 1) Find project root: go up from notebooks/ to repo root
NOTEBOOK_DIR = Path.cwd()                      # when you open Jupyter from project root
if NOTEBOOK_DIR.name == "notebooks":
    PROJECT_ROOT = NOTEBOOK_DIR.parent
else:
    # if you're not inside notebooks/, still handle it safely:
    # assume repo root is current dir or walk up until "notebooks" exists
    p = NOTEBOOK_DIR
    while p != p.parent and not (p / "notebooks").exists():
        p = p.parent
    PROJECT_ROOT = p

# 2) Auto-detect dataset location (supports a couple common layouts)
CANDIDATES = [
    PROJECT_ROOT / "dataset" / "archive",
    PROJECT_ROOT / "dataset",
    PROJECT_ROOT / "data" / "ravdess",
    PROJECT_ROOT / "ravdess",
]

RAVDESS_ROOT = next((c for c in CANDIDATES if c.exists()), None)
if RAVDESS_ROOT is None:
    raise FileNotFoundError(
        f"Could not find RAVDESS dataset folder. Checked: {CANDIDATES}\n"
        f"Project root detected as: {PROJECT_ROOT}"
    )

# 3) Output directory
OUTPUT_DIR = PROJECT_ROOT / "notebooks" / "data"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# 4) Collect all wav files (recursive)
wav_files = [str(p) for p in RAVDESS_ROOT.rglob("*.wav")]

print("Project root:", PROJECT_ROOT)
print("RAVDESS_ROOT:", RAVDESS_ROOT)
print("OUTPUT_DIR:", OUTPUT_DIR)
print("Total WAV files:", len(wav_files))
print("Example file:", wav_files[0] if wav_files else "None")



Project root: /Users/malakmaher/Documents/GitHub/Emotion-Recognition
RAVDESS_ROOT: /Users/malakmaher/Documents/GitHub/Emotion-Recognition/dataset/archive
OUTPUT_DIR: /Users/malakmaher/Documents/GitHub/Emotion-Recognition/notebooks/data
Total WAV files: 2781
Example file: /Users/malakmaher/Documents/GitHub/Emotion-Recognition/dataset/archive/Actor_16/03-01-05-01-02-01-16.wav


In [40]:
EMOTION_LABELS = [
    "neutral", "calm", "happy", "sad",
    "angry", "fearful", "disgust", "surprised"
]

emotion_map = {
    1: 0,  # neutral
    2: 1,  # calm
    3: 2,  # happy
    4: 3,  # sad
    5: 4,  # angry
    6: 5,  # fearful
    7: 6,  # disgust
    8: 7   # surprised
}


In [41]:
import os

wav_files = []
for root, _, files in os.walk(str(RAVDESS_ROOT)):
    for f in files:
        if f.endswith(".wav"):
            wav_files.append(os.path.join(root, f))

print("Total WAV files:", len(wav_files))
print("Example:", wav_files[0] if wav_files else "None")


Total WAV files: 2781
Example: /Users/malakmaher/Documents/GitHub/Emotion-Recognition/dataset/archive/Actor_16/03-01-05-01-02-01-16.wav


In [42]:
X = []
y = []

for wav_path in tqdm(wav_files):
    filename = os.path.basename(wav_path)
    emotion_code = int(filename.split("-")[2])
    label = emotion_map[emotion_code]

    mel = wav_to_logmel(wav_path)  # (128,128,1)
    X.append(mel)
    y.append(label)

X = np.array(X)
y = np.array(y)

print("X shape:", X.shape)
print("y shape:", y.shape)


100%|██████████| 2781/2781 [00:16<00:00, 167.64it/s]

X shape: (2781, 128, 128, 1)
y shape: (2781,)


In [43]:
mean = X.mean()
std = X.std()

print("Mean:", mean)
print("Std:", std)


Mean: 0.12280687
Std: 0.21521616


In [44]:
X = (X - mean) / std

np.save(os.path.join(OUTPUT_DIR, "X.npy"), X)
np.save(os.path.join(OUTPUT_DIR, "y.npy"), y)

print("Saved X.npy and y.npy")


Saved X.npy and y.npy
